# Course project

This notebook includes the steps to optimize the capacity and dispatch of generators in a stylized energy system. It has been prepared to serve as a tutorial for a simple PyPSA model representing the energy system in one country, city or region. 
 

For the (optional) project of the course [Integrated Energy Grids](https://kurser.dtu.dk/course/2024-2025/46770?menulanguage=en) you need to deliver a report including the sections described at the end of this notebook.

Please, review the [PyPSA tutorial](https://martavp.github.io/integrated-energy-grids/intro-pypsa.html) before starting this project.

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install `pandas` and `pypsa` by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas pypsa
```
:::

In [3]:
import pandas as pd
import pypsa
import urllib3

urllib3.disable_warnings()

In [4]:
# Load Technology data
year = 2025
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{year}.csv"
costs = pd.read_csv(url, index_col=[0, 1])

costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
costs.unit = costs.unit.str.replace("/kW", "/MW")

defaults = {
    "FOM": 0,
    "VOM": 0,
    "efficiency": 1,
    "fuel": 0,
    "investment": 0,
    "lifetime": 25,
    "CO2 intensity": 0,
    "discount rate": 0.07,
}
costs = costs.value.unstack().fillna(defaults)

costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["OCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]

costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]


We start by creating the network. In this example, the country is modelled as a single node, so the network includes only one bus.

We select the year 2015 and set the hours in that year as snapshots.

We select a country, in this case Denmark (DNK), and add one node (electricity bus) to the network.

In [5]:
network = pypsa.Network()
hours_in_2015 = pd.date_range('2015-01-01 00:00Z',
                              '2015-12-31 23:00Z',
                              freq='h')

network.set_snapshots(hours_in_2015.values)

network.add("Bus",
            "electricity bus")

network.snapshots

DatetimeIndex(['2015-01-01 00:00:00', '2015-01-01 01:00:00',
               '2015-01-01 02:00:00', '2015-01-01 03:00:00',
               '2015-01-01 04:00:00', '2015-01-01 05:00:00',
               '2015-01-01 06:00:00', '2015-01-01 07:00:00',
               '2015-01-01 08:00:00', '2015-01-01 09:00:00',
               ...
               '2015-12-31 14:00:00', '2015-12-31 15:00:00',
               '2015-12-31 16:00:00', '2015-12-31 17:00:00',
               '2015-12-31 18:00:00', '2015-12-31 19:00:00',
               '2015-12-31 20:00:00', '2015-12-31 21:00:00',
               '2015-12-31 22:00:00', '2015-12-31 23:00:00'],
              dtype='datetime64[ns]', name='snapshot', length=8760, freq=None)

The demand is represented by the historical electricity demand in 2015 with hourly resolution.

The file with historical hourly electricity demand for every European country is available in the data folder.

The electricity demand time series were obtained from ENTSOE through the very convenient compilation carried out by the Open Power System Data (OPSD). https://data.open-power-system-data.org/time_series/

In [6]:
# load electricity demand data
df_elec = pd.read_csv('data/electricity_demand.csv', sep=';', index_col=0) # in MWh
df_elec.index = pd.to_datetime(df_elec.index) #change index to datatime
country='IRL'
print(df_elec[country].head())

utc_time
2015-01-01 00:00:00+00:00    2651.0
2015-01-01 01:00:00+00:00    2412.0
2015-01-01 02:00:00+00:00    2256.0
2015-01-01 03:00:00+00:00    2170.0
2015-01-01 04:00:00+00:00    2136.0
Name: IRL, dtype: float64


In [7]:
# add load to the bus
network.add("Load",
            "load",
            bus="electricity bus",
            p_set=df_elec[country].values)

Index(['load'], dtype='object')

Print the load time series to check that it has been properly added (you should see numbers and not 'NaN')

In [8]:
network.loads_t.p_set

Load,load
snapshot,
2015-01-01 00:00:00,2651.0
2015-01-01 01:00:00,2412.0
2015-01-01 02:00:00,2256.0
2015-01-01 03:00:00,2170.0
2015-01-01 04:00:00,2136.0
...,...
2015-12-31 19:00:00,3728.0
2015-12-31 20:00:00,3461.0
2015-12-31 21:00:00,3195.0


In the optimization, we will minimize the annualized system costs.

We will need to annualize the cost of every generator, we build a function to do it.

In [9]:
def annuity(n,r):
    """ Calculate the annuity factor for an asset with lifetime n years and
    discount rate  r """

    if r > 0:
        return r/(1. - 1./(1.+r)**n)
    else:
        return 1/n


In [10]:
annuity = costs.apply(lambda x: annuity(x["discount rate"], x["lifetime"]), axis=1)
costs["capital_cost"] = (annuity + costs["FOM"] / 100) * costs["investment"]

In [11]:
annuity

technology
Ammonia cracker                       122.591755
BEV Bus city                           73.014521
BEV Coach                              73.014521
BEV Truck Semi-Trailer max 50 tons     66.815854
BEV Truck Solo max 26 tons             80.278248
                                         ...    
uranium                               122.591755
waste CHP                             122.591755
waste CHP CC                          122.591755
water tank charger                    122.591755
water tank discharger                 122.591755
Length: 272, dtype: float64

We include solar PV and onshore wind generators.

The capacity factors representing the availability of those generators for every European country can be downloaded from the following repositories (select 'optimal' for PV and onshore for wind).

https://zenodo.org/record/3253876#.XSiVOEdS8l0

https://zenodo.org/record/2613651#.XSiVOkdS8l0

We include also Open Cycle Gas Turbine (OCGT) generators

The cost assumed for the generators are the same as in Table 1 in the paper https://doi.org/10.1016/j.enconman.2019.111977 (open version:  https://arxiv.org/pdf/1906.06936.pdf)

In [12]:
# add the different carriers, only gas emits CO2
network.add("Carrier", "gas", co2_emissions= costs.at["gas", "CO2 intensity"]) # in t_CO2/MWh_th
network.add("Carrier", "onshorewind")
network.add("Carrier", "solar")
network.add("Carrier", "coal", co2_emissions = costs.at["coal", "CO2 intensity"]) # in t_CO2/MWh_th
network.add("Carrier", "biomass", co2_emissions = costs.at["biomass", "CO2 intensity"]) # in t_CO2/MWh_th
network.add("Carrier", "oil", co2_emissions = costs.at["oil", "CO2 intensity"]) # in t_CO2/MWh_th
network.add("Carrier", "offshorewind")

# add onshore wind generator
df_onshorewind = pd.read_csv('data/onshore_wind_1979-2017.csv', sep=';', index_col=0)
df_onshorewind.index = pd.to_datetime(df_onshorewind.index)
CF_wind = df_onshorewind[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
network.add("Generator",
            "onshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="onshorewind",
            p_nom_max=4810-25.2, # maximum capacity can be limited due to environmental constraints. Based on electricity maps - minus the off shore wind capacity
            capital_cost = costs.at["onwind", "capital_cost"], # in €/MW
            marginal_cost = 0,
            p_max_pu = CF_wind.values)

# add solar PV generator
df_solar = pd.read_csv('data/pv_optimal.csv', sep=';', index_col=0)
df_solar.index = pd.to_datetime(df_solar.index)
CF_solar = df_solar[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
network.add("Generator",
            "solar",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="solar",
            p_nom_max = 740, # maximum capacity can be limited due to environmental constraints Based on electricity maps
            capital_cost = costs.at["solar", "capital_cost"], # in €/MW
            marginal_cost = 0,
            p_max_pu = CF_solar.values)

# add OCGT (Open Cycle Gas Turbine) generator
network.add("Generator",
            "OCGT",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="gas",
            p_nom_max=4270, #based on electricity maps
            capital_cost = costs.at["OCGT", "capital_cost"], #in €/MW
            marginal_cost = costs.at["OCGT", "marginal_cost"])

# add Coal generator
network.add("Generator",
            "Coal",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="coal",
            p_nom_max=920, #based on electricity maps
            capital_cost = costs.at["coal", "capital_cost"], #in €/MW
            marginal_cost = costs.at["coal", "marginal_cost"])

# add biomass generator
network.add("Generator",
            "Biomass",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="biomass",
            p_nom_max=70, #based on electricity maps 
            capital_cost = costs.at["biomass", "capital_cost"], #in €/MW
            marginal_cost = costs.at["biomass", "marginal_cost"])

# add oil generator
network.add("Generator",
            "Oil",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="oil",
            p_nom_max=830, #based on electricity maps
            capital_cost = costs.at["oil", "capital_cost"], #in €/MW
            marginal_cost = costs.at["oil", "marginal_cost"])

# add offshore wind generator
df_offshorewind = pd.read_csv('data/offshore_wind_1979-2017.csv', sep=';', index_col=0)
df_offshorewind.index = pd.to_datetime(df_offshorewind.index)
CF_offshorewind = df_offshorewind[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
network.add("Generator",
            "offshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="offshorewind",
            p_nom_max=25.2, # maximum capacity can be limited due to environmental constraints. #Ireland doesnt have offshore wind today, but are proposing to install 5GW soon
            capital_cost = costs.at["offwind", "capital_cost"], # in €/MW
            marginal_cost = 0,
            p_max_pu = CF_offshorewind.values)




Index(['offshorewind'], dtype='object')

Print the generator Capacity factor time series to check that it has been properly added (you should see numbers and not 'NaN')

In [13]:
network.generators_t.p_max_pu

Generator,offshorewind,onshorewind,solar
snapshot,,,
2015-01-01 00:00:00,0.597,0.472,0.0
2015-01-01 01:00:00,0.507,0.435,0.0
2015-01-01 02:00:00,0.412,0.449,0.0
2015-01-01 03:00:00,0.562,0.524,0.0
2015-01-01 04:00:00,0.739,0.593,0.0
...,...,...,...
2015-12-31 19:00:00,0.324,0.156,0.0
2015-12-31 20:00:00,0.179,0.162,0.0
2015-12-31 21:00:00,0.146,0.155,0.0


# A)

We find the optimal solution using Gurobi as solver.

In this case, we are optimizing the installed capacity and dispatch of every generator to minimize the total system cost.

In [14]:
network.optimize(solver_name='gurobi')

Index(['electricity bus'], dtype='object', name='Bus')
Index(['electricity bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 2/2 [00:00<00:00, 39.17it/s]
INFO:linopy.io: Writing time: 0.35s


Set parameter Username


INFO:gurobipy:Set parameter Username


Academic license - for non-commercial use only - expires 2026-02-12


INFO:gurobipy:Academic license - for non-commercial use only - expires 2026-02-12


...

Solved in 40655 iterations and 0.37 seconds (0.63 work units)


INFO:gurobipy:Solved in 40655 iterations and 0.37 seconds (0.63 work units)


Optimal objective  2.597451167e+11


INFO:gurobipy:Optimal objective  2.597451167e+11
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 61327 primals, 131414 duals
Objective: 2.60e+11
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper were not assigned to the network.


('ok', 'optimal')

The message ('ok' , 'optimal") indicates that the optimizer has found an optimal solution.

The total cost can be read from the network objetive.

In [15]:
print(network.objective/1000000) #in 10^6 €

259745.11669723087


The cost per MWh of electricity produced can also be calculated.

In [16]:
print(network.objective/network.loads_t.p.sum()) # EUR/MWh

Load
load    9775.045783
dtype: float64


The optimal capacity for every generator can be shown.

In [17]:
network.generators.p_nom_opt # in MW

Generator
onshorewind        0.0
solar              0.0
OCGT            3832.0
Coal               0.0
Biomass            0.0
Oil              830.0
offshorewind       0.0
Name: p_nom_opt, dtype: float64

We can plot now the dispatch of every generator during the first week of the year and the electricity demand.
We import the matplotlib package which is very useful to plot results.

We can also plot the electricity mix.

In [18]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.loads_t.p['load'][:96], mode='lines', name='demand', line=dict(color='black')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['onshorewind'][:96], mode='lines', name='onshore wind', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['solar'][:96], mode='lines', name='solar', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['OCGT'][:96], mode='lines', name='gas (OCGT)', line=dict(color='brown')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['Coal'][:96], mode='lines', name='coal', line=dict(color='grey')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['Biomass'][:96], mode='lines', name='biomass', line=dict(color='green')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['Oil'][:96], mode='lines', name='oil', line=dict(color='black')))
fig.add_trace(go.Scatter(x=network.snapshots[:96], y=network.generators_t.p['offshorewind'][:96], mode='lines', name='offshore wind', line=dict(color='purple')))

fig.update_layout(title='Electricity Dispatch Week 1 2015',
                  xaxis_title='Time',
                  yaxis_title='Power (MW)',
                  legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

In [19]:
import plotly.graph_objects as go

fig = go.Figure()

# Define the start and end of the first week of July
start_date = '2015-07-01'
end_date = '2015-07-07'

# Filter the snapshots for the first week of July
snapshots_july = network.snapshots[(network.snapshots >= start_date) & (network.snapshots < end_date)]

fig.add_trace(go.Scatter(x=snapshots_july, y=network.loads_t.p['load'][snapshots_july], mode='lines', name='demand', line=dict(color='black')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['onshorewind'][snapshots_july], mode='lines', name='onshore wind', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['solar'][snapshots_july], mode='lines', name='solar', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['OCGT'][snapshots_july], mode='lines', name='gas (OCGT)', line=dict(color='brown')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['Coal'][snapshots_july], mode='lines', name='coal', line=dict(color='grey')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['Biomass'][snapshots_july], mode='lines', name='biomass', line=dict(color='green')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['Oil'][snapshots_july], mode='lines', name='oil', line=dict(color='black')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network.generators_t.p['offshorewind'][snapshots_july], mode='lines', name='offshore wind', line=dict(color='purple')))

fig.update_layout(title='Electricity Dispatch Week 1 July 2015',
                  xaxis_title='Time',
                  yaxis_title='Power (MW)',
                  legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

In [20]:
import plotly.graph_objects as go

labels = ['onshore wind', 'solar', 'gas (OCGT)', 'coal', 'biomass', 'oil', 'offshore wind']
sizes = [network.generators_t.p['onshorewind'].sum(),
                 network.generators_t.p['solar'].sum(),
                 network.generators_t.p['OCGT'].sum(),
                 network.generators_t.p['Coal'].sum(),
                 network.generators_t.p['Biomass'].sum(),
                 network.generators_t.p['Oil'].sum(),
                 network.generators_t.p['offshorewind'].sum()]

colors = ['blue', 'orange', 'brown', 'grey', 'green', 'black', 'purple']

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes, marker=dict(colors=colors))])

fig.update_layout(title='Electricity mix week 1 2015')

fig.show()


In [21]:
import plotly.graph_objects as go

# Calculate the duration curve for the demand
demand_sorted = network.loads_t.p['load'].sort_values(ascending=False).values

# Create the duration curve plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(demand_sorted))), y=demand_sorted, mode='lines', name='Demand', line=dict(color='black')))

fig.update_layout(title='Duration Curve',
                  xaxis_title='Hours',
                  yaxis_title='Power (MW)',
                  legend=dict(x=0, y=1, traceorder='normal'))

fig.show()


# B)

We can add a global CO2 constraint and solve again.

In [22]:

# Make a copy of the network
network_CO2 = network

# Add the CO2 constraint to the copied network
co2_limit = 2000000  # tonCO2
network_CO2.add("GlobalConstraint",
                 "co2_limit",
                 type="primary_energy",
                 carrier_attribute="co2_emissions",
                 sense="<=",
                 constant=co2_limit)

# Optimize the copied network
network_CO2.optimize(solver_name='gurobi')

Index(['electricity bus'], dtype='object', name='Bus')
Index(['electricity bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 2/2 [00:00<00:00, 39.92it/s]
INFO:linopy.io: Writing time: 0.35s


Set parameter Username


INFO:gurobipy:Set parameter Username


Academic license - for non-commercial use only - expires 2026-02-12


INFO:gurobipy:Academic license - for non-commercial use only - expires 2026-02-12


...

Solved in 25636 iterations and 0.25 seconds (0.45 work units)


INFO:gurobipy:Solved in 25636 iterations and 0.25 seconds (0.45 work units)


Infeasible or unbounded model


INFO:gurobipy:Infeasible or unbounded model
INFO:linopy.solvers:Unable to save solution file. Raised error: Unable to retrieve attribute 'X'
Status: warning
Termination condition: infeasible_or_unbounded
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 4



('warning', 'infeasible_or_unbounded')

In [23]:
#Total cost
print(network_CO2.objective/1000000) #in 10^6 €

259745.11669723087


In [24]:
#cost pr MWh
print(network.objective/network.loads_t.p.sum()) # EUR/MWh

Load
load    9775.045783
dtype: float64


In [25]:
network_CO2.generators.p_nom_opt #in MW

Generator
onshorewind        0.0
solar              0.0
OCGT            3832.0
Coal               0.0
Biomass            0.0
Oil              830.0
offshorewind       0.0
Name: p_nom_opt, dtype: float64

In [26]:
#print Co2 emissions from coal
print(network_CO2.generators_t.p['Coal'].sum() * costs.at["coal", "CO2 intensity"])
print(network_CO2.generators_t.p['OCGT'].sum() * costs.at["gas", "CO2 intensity"])

print(network_CO2.global_constraints.constant)

0.0
5231704.104
GlobalConstraint
co2_limit    2000000.0
Name: constant, dtype: float64


In [27]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.loads_t.p['load'][:96], mode='lines', name='demand', line=dict(color='black')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['onshorewind'][:96], mode='lines', name='onshore wind', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['solar'][:96], mode='lines', name='solar', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['OCGT'][:96], mode='lines', name='gas (OCGT)', line=dict(color='brown')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['Coal'][:96], mode='lines', name='coal', line=dict(color='grey')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['Biomass'][:96], mode='lines', name='biomass', line=dict(color='green')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['Oil'][:96], mode='lines', name='oil', line=dict(color='black')))
fig.add_trace(go.Scatter(x=network_CO2.snapshots[:96], y=network_CO2.generators_t.p['offshorewind'][:96], mode='lines', name='offshore wind', line=dict(color='purple')))

fig.update_layout(title='Electricity Dispatch',
                  xaxis_title='Time',
                  yaxis_title='Power (MW)',
                  legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

In [28]:
import plotly.graph_objects as go

fig = go.Figure()

# Define the start and end of the first week of July
start_date = '2015-07-01'
end_date = '2015-07-07'

# Filter the snapshots for the first week of July
snapshots_july = network_CO2.snapshots[(network_CO2.snapshots >= start_date) & (network_CO2.snapshots < end_date)]

fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.loads_t.p['load'][snapshots_july], mode='lines', name='demand', line=dict(color='black')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['onshorewind'][snapshots_july], mode='lines', name='onshore wind', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['solar'][snapshots_july], mode='lines', name='solar', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['OCGT'][snapshots_july], mode='lines', name='gas (OCGT)', line=dict(color='brown')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['Coal'][snapshots_july], mode='lines', name='coal', line=dict(color='grey')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['Biomass'][snapshots_july], mode='lines', name='biomass', line=dict(color='green')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['Oil'][snapshots_july], mode='lines', name='oil', line=dict(color='black')))
fig.add_trace(go.Scatter(x=snapshots_july, y=network_CO2.generators_t.p['offshorewind'][snapshots_july], mode='lines', name='offshore wind', line=dict(color='purple')))

fig.update_layout(title='Electricity Dispatch Week 1 July 2015',
                  xaxis_title='Time',
                  yaxis_title='Power (MW)',
                  legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

In [29]:
import plotly.graph_objects as go

labels = ['onshore wind', 'solar', 'gas (OCGT)', 'coal', 'biomass', 'oil', 'offshore wind']
sizes = [network_CO2.generators_t.p['onshorewind'].sum(),
                 network_CO2.generators_t.p['solar'].sum(),
                 network_CO2.generators_t.p['OCGT'].sum(),
                 network_CO2.generators_t.p['Coal'].sum(),
                 network_CO2.generators_t.p['Biomass'].sum(),
                 network_CO2.generators_t.p['Oil'].sum(),
                 network_CO2.generators_t.p['offshorewind'].sum()]

colors = ['blue', 'orange', 'brown', 'grey', 'green', 'black', 'purple']

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes, marker=dict(colors=colors))])

fig.update_layout(title='Electricity mix')

fig.show()


In [30]:
import plotly.graph_objects as go

# Calculate the duration curve for each generator type
generators = ['onshorewind', 'solar', 'OCGT', 'Coal', 'Biomass', 'Oil', 'offshorewind']
duration_curves = {gen: network.generators_t.p[gen].sort_values(ascending=False).values for gen in generators}

# Create the duration curve plot
fig = go.Figure()

for gen in generators:
    fig.add_trace(go.Scatter(x=list(range(len(duration_curves[gen]))), y=duration_curves[gen], mode='lines', name=gen))

fig.update_layout(title='Duration Curve by Generator Type',
                  xaxis_title='Hours',
                  yaxis_title='Power (MW)',
                  legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

# PROJECT INSTRUCTIONS


Based on the previous example, you are asked to carry out the following tasks:

A. Choose a different country/region/city/system and calculate the optimal capacities for renewable and non-renewable generators. You can add as many technologies as you want. Remember to provide a reference for the cost assumptions. Plot the dispatch time series for a week in summer and winter. Plot the annual electricity mix. Use the duration curves or the capacity factor to investigate the contribution of different technologies.

B. Investigate how sensitive the optimum capacity mix is to the global CO2 constraint. E.g., plot the generation mix as a function of the CO2 constraint that you impose. Search for the CO2 emissions in your country (today or in 1990) and refer to the emissions allowance to that historical data.

C. Investigate how sensitive your results are to the interannual variability of solar and wind generation. Plot the average capacity and variability obtained for every generator using different weather years.

D. Add some storage technology/ies and investigate how they behave and what their impact is on the optimal system configuration. Discuss what strategies is your system using to balance the renewable generation at different time scales (intraday, seasonal, etc.)

E. Select one target for decarbonization (i.e., one CO2 allowance limit). What is the CO2 price required to achieve that decarbonization level? Search for information on the existing CO2 tax in your country (if any) and discuss your results.

F. Connect your country with, at least, two neighbouring countries. You can connect them using HVAC lines, HVDC links or gas pipelines. Use a linear representation of power flow or gas flow. 
You can assume that the generation capacities in the neighbouring countries are fixed or optimize the whole system. You can also include fixed interconnection capacities or optimize them with the generators' capacities. Discuss your results.

G. Connect the electricity sector with, at least, another sector( e.g. heating or transport), and co-optimize all the sectors. Discuss your results.

H. Finally, select one topic that is under discussion in your region. Design and implement some experiment to obtain relevant information regarding that topic. E.g.

- What are the consequences if Denmark decides not to install more onshore wind?

- Would it be more expensive if France decides to close its nuclear power plants?

- What will be the main impacts of the Viking link?

- How does gas scarcity impact the optimal system configuration?

**Write a short report (maximum length 10 pages) in groups of 4 students including your main findings.**

# Hints

_HINT 1: You can add a link with the following code_

The efficiency will be 1 if you are connecting two countries and different from one if, for example, you are connecting the electricity bus to the heating bus using a heat pump.
Setting p_min_pu=-1 makes the link reversible.


In [31]:
network.add("Link",
             'country a - country b',
             bus0="electricity bus country a",
             bus1="electricity bus country b",
             p_nom_extendable=True, # capacity is optimised
             p_min_pu=-1,
             length=600, # length (in km) between country a and country b
             capital_cost=400*600) # capital cost * length

Index(['country a - country b'], dtype='object')
Index(['country a - country b'], dtype='object')


Index(['country a - country b'], dtype='object')

_HINT 2: You can check the KKT multiplier associated with the constraint with the following code_

In [32]:
print(network.global_constraints.constant) # CO2 limit (constant in the constraint)
print(network.global_constraints.mu) # CO2 price (Lagrance multiplier in the constraint)

GlobalConstraint
co2_limit    2000000.0
Name: constant, dtype: float64
GlobalConstraint
co2_limit    0.0
Name: mu, dtype: float64


_HINT 3: You can add a H2 store connected to the electricity bus via an electrolyzer and a fuel cell with the following code_

In [33]:
#Create a new carrier
network.add("Carrier",
              "H2")

#Create a new bus
network.add("Bus",
          "H2",
          carrier = "H2")

#Connect the store to the bus
network.add("Store",
          "H2 Tank",
          bus = "H2",
          e_nom_extendable = True,
          e_cyclic = True,
          capital_cost = annuity(25, 0.07)*57000*(1+0.011))

#Add the link "H2 Electrolysis" that transport energy from the electricity bus (bus0) to the H2 bus (bus1)
#with 80% efficiency
network.add("Link",
          "H2 Electrolysis",
          bus0 = "electricity bus",
          bus1 = "H2",
          p_nom_extendable = True,
          efficiency = 0.8,
          capital_cost = annuity(25, 0.07)*600000*(1+0.05))

#Add the link "H2 Fuel Cell" that transports energy from the H2 bus (bus0) to the electricity bus (bus1)
#with 58% efficiency
network.add("Link",
          "H2 Fuel Cell",
          bus0 = "H2",
          bus1 = "electricity bus",
          p_nom_extendable = True,
          efficiency = 0.58,
          capital_cost = annuity(10, 0.07)*1300000*(1+0.05))


TypeError: 'Series' object is not callable

_HINT 4: You can get inspiration for plotting the flows in the network in the following example_

https://pypsa.readthedocs.io/en/latest/examples/flow-plot.html